In [29]:
import os
import json
import pydicom
import tkinter as tk
from tkinter import ttk, messagebox
from collections import defaultdict
import subprocess
from pathlib import Path

In [ ]:
def extract_series_info(dicom_dir):
    series_info = defaultdict(list)
    for root, _, files in os.walk(dicom_dir):
        for file in files:
            if file.endswith('.IMA'):
                dicom_path = os.path.join(root, file)
                try:
                    dicom = pydicom.dcmread(dicom_path)
                    series_desc = dicom.SeriesDescription
                    series_info[series_desc].append(dicom_path)
                except Exception as e:
                    print(f"Could not read {dicom_path}: {e}")
    return series_info

def infer_bids_mapping(series_info):
    config = {"descriptions": []}
    for series_desc in series_info:
        if "T1" in series_desc:
            data_type = "anat"
            modality_label = "T1w"
            custom_labels = "acq-mprage"
        elif "Rest" in series_desc or "rest" in series_desc:
            data_type = "func"
            modality_label = "bold"
            custom_labels = "task-rest"
        else:
            # Add more rules as needed
            continue

        config["descriptions"].append({
            "dataType": data_type,
            "modalityLabel": modality_label,
            "customLabels": custom_labels,
            "criteria": {
                "SeriesDescription": series_desc
            }
        })
    return config

def write_config_file(config, output_path):
    with open(output_path, 'w') as f:
        json.dump(config, f, indent=4)

def main(dicom_dir, output_path):
    series_info = extract_series_info(dicom_dir)
    config = infer_bids_mapping(series_info)
    write_config_file(config, output_path)

# Define paths
dicom_dir = '/zi/flstorage/group_klips/data/data/fMRI_workgroup/data/RAWDATA/TEST_SEMINAR_HD/'
config_output_path = 'config.json'

# Run the script
main(dicom_dir, config_output_path)

In [5]:
dicom_dir = '/zi/flstorage/group_klips/data/data/fMRI_workgroup/data/RAWDATA/TEST_SEMINAR_HD/'
series_info = defaultdict(list)
for root, _, files in os.walk(dicom_dir):
    for file in files:
        if file.endswith('.IMA'):
            dicom_path = os.path.join(root, file)
            try:
                dicom = pydicom.dcmread(dicom_path)
                series_desc = dicom.SeriesDescription
                series_info[series_desc].append(dicom_path)
            except Exception as e:
                print(f"Could not read {dicom_path}: {e}")

In [13]:
series_info.keys()

dict_keys(['t1_mprage_sag_p2_iso', 'AAHead_Scout_32ch-head-coil', 'ep2d_mid', 'AAHead_Scout_32ch-head-coil_MPR_sag', 'AAHead_Scout_32ch-head-coil_MPR_tra', 'AAHead_Scout_32ch-head-coil_MPR_cor'])

In [17]:
dicom_dir = '/zi/flstorage/group_klips/data/data/fMRI_workgroup/data/RAWDATA/TEST_SEMINAR_HD/'
config_output_path = 'config.json'
## get DICOM info
def extract_series_info(dicom_dir):
    series_info = defaultdict(list)
    for root, _, files in os.walk(dicom_dir):
        for file in files:
            if file.endswith('.IMA'):
                dicom_path = os.path.join(root, file)
                try:
                    dicom = pydicom.dcmread(dicom_path)
                    series_desc = dicom.SeriesDescription
                    series_info[series_desc].append(dicom_path)
                except Exception as e:
                    print(f"Could not read {dicom_path}: {e}")
    return series_info # return the dictionary with all filenames and different filetypes
##
def infer_bids_mapping(series_info):
    config = {"descriptions": []}
    for series_desc in series_info:
        if "T1" in series_desc or "t1" in series_desc:
            data_type = "anat"
            modality_label = "T1w"
            custom_labels = "acq-mprage"
        elif "Rest" in series_desc or "rest" in series_desc:
            data_type = "func"
            modality_label = "bold"
            custom_labels = "task-rest"
        elif "ep2d_" in series_desc:
            data_type = "func"
            modality_label = "bold"
            custom_labels = "task-exp"
        else:
            # Add more rules as needed
            continue

        config["descriptions"].append({
            "datatype": data_type,
            "suffix": modality_label,
            "customLabels": custom_labels,
            "criteria": {
                "SeriesDescription": "*"+series_desc+"*"
            }
        })
    return config
##
def write_config_file(config, output_path):
    with open(output_path, 'w') as f:
        json.dump(config, f, indent=4)
##
def main(dicom_dir, output_path):
    series_info = extract_series_info(dicom_dir)
    config = infer_bids_mapping(series_info)
    write_config_file(config, output_path)

# Run the script
main(dicom_dir, config_output_path)

In [28]:
dicom_dir = '/zi/flstorage/group_klips/data/data/fMRI_workgroup/data/RAWDATA/TEDDS/'
config_output_path = 'config2.json'

class PlaceholderEntry(tk.Entry):
    def __init__(self, master=None, placeholder="PLACEHOLDER", color='grey'):
        super().__init__(master)

        self.placeholder = placeholder
        self.placeholder_color = color
        self.default_fg_color = self['fg']

        self.bind("<FocusIn>", self.foc_in)
        self.bind("<FocusOut>", self.foc_out)

        self.put_placeholder()

    def put_placeholder(self):
        self.insert(0, self.placeholder)
        self['fg'] = self.placeholder_color

    def foc_in(self, *args):
        if self['fg'] == self.placeholder_color:
            self.delete('0', 'end')
            self['fg'] = self.default_fg_color

    def foc_out(self, *args):
        if not self.get():
            self.put_placeholder()

class SeriesInfoGUI:
    def __init__(self, master, series_info, output_path):
        self.master = master
        self.series_info = series_info
        self.output_path = output_path
        self.entries = {}
        
        self.master.title("DICOM Series Information")
        
        row = 0
        tk.Label(master, text="Series Description").grid(row=row, column=0)
        tk.Label(master, text="Data Type").grid(row=row, column=1)
        tk.Label(master, text="BIDS Label").grid(row=row, column=2)
        tk.Label(master, text="Custom Labels").grid(row=row, column=3)
        
        for series_desc in series_info:
            row += 1
            tk.Label(master, text=series_desc).grid(row=row, column=0)
            self.entries[series_desc] = {
                "dataType": PlaceholderEntry(master, placeholder="e.g., anat, func"),
                "modalityLabel": PlaceholderEntry(master, placeholder="e.g., T1w, bold, "),
                "customLabels": PlaceholderEntry(master, placeholder="e.g., acq-mprage, task-rest")
            }
            self.entries[series_desc]["dataType"].grid(row=row, column=1)
            self.entries[series_desc]["modalityLabel"].grid(row=row, column=2)
            self.entries[series_desc]["customLabels"].grid(row=row, column=3)
        
        row += 1
        tk.Button(master, text="Save Config", command=self.save_config).grid(row=row, column=1, columnspan=2)

    def save_config(self):
        config = {"descriptions": []}
        for series_desc, entries in self.entries.items():
            data_type = entries["dataType"].get()
            modality_label = entries["modalityLabel"].get()
            custom_labels = entries["customLabels"].get()
            if data_type == "" or "e.g." in data_type:
                continue
            elif data_type and modality_label:
                config["descriptions"].append({
                    "datatype": data_type if data_type != "e.g., anat, func" else "",
                    "suffix": modality_label if modality_label != "e.g., T1w, bold" else "",
                    "custom_entities": custom_labels if custom_labels != "e.g., acq-mprage, task-rest" else "",
                    "criteria": {
                        "SeriesDescription": series_desc
                    }
                })
        
        with open(self.output_path, 'w') as f:
            json.dump(config, f, indent=4)
        
        messagebox.showinfo("Info", "Configuration saved successfully!")

def main(dicom_dir, output_path):
    series_info = extract_series_info(dicom_dir)
    root = tk.Tk()
    app = SeriesInfoGUI(root, series_info, output_path)
    root.mainloop()

# Run the script
main(dicom_dir, config_output_path)

In [35]:
# --clobber --force_dcm2bids

# Define the paths
dicom_path = '/zi/flstorage/group_klips/data/data/fMRI_workgroup/data/RAWDATA/TEDDS/'
output_path = 'bids_test2'
config_file = 'config2.json'

# Get all subject folder names
sub_fld = [folder.name for folder in Path(dicom_path).iterdir() if folder.is_dir()]

# Create the output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Now run everything in a loop
for subject_id in sub_fld:
    # Construct the dcm2bids command
    command = [
        'dcm2bids',
        '-d', dicom_path,
        '-p', subject_id,
        '-c', config_file,
        '-o', output_path,
        '-- clobber',
        '--force_dcm2bids'
    ]
    
    # Run the command
    result = subprocess.run(command, capture_output=True, text=True)

# # Check the result
# if result.returncode == 0:
#     print("dcm2bids ran successfully")
# else:
#     print("Error running dcm2bids:", result.stderr)

['dcm2bids', '-d', '/zi/flstorage/group_klips/data/data/fMRI_workgroup/data/RAWDATA/TEDDS/', '-p', 'OCRT_05', '-c', 'config2.json', '-o', 'bids_test2', '-- clobber', '--force_dcm2bids']


FileNotFoundError: [Errno 2] No such file or directory: 'dcm2bids'


CondaError: Run 'conda init' before 'conda activate'

